In [1]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Set Paths
# Replace 'path_to_zip_in_drive' with the actual path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/PADDY IMAGES.zip'  # Example: 'dataset.zip'
extract_to = '/content/'  # Destination folder in Colab

# Step 3: Extract the Zip File
import zipfile
import os

# Create the extraction folder if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete!")
print(f"Files extracted to: {extract_to}")

# Optional: List extracted files
print("Extracted files:")
print(os.listdir(extract_to))


Mounted at /content/drive
Extraction complete!
Files extracted to: /content/
Extracted files:
['.config', 'drive', 'PADDY IMAGES', 'sample_data']


In [13]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Define input and output directories
input_dir = "/content/PADDY IMAGES"  # Path to your dataset
output_dir = "/content/preprocessed"  # Output folder for preprocessed images

# Preprocessing parameters
image_size = (299, 299)  # Input size for Xception model

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Function to preprocess and save images
def preprocess_and_save(input_path, output_path):
    try:
        # Read the image
        image = cv2.imread(input_path)
        if image is None:
            print(f"Error reading image: {input_path}")
            return

        # Resize the image
        resized_image = cv2.resize(image, image_size)

        # Normalize the image (scale pixel values to [0, 1])
        normalized_image = resized_image / 255.0

        # Convert back to uint8 for saving (scale [0, 1] to [0, 255])
        output_image = (normalized_image * 255).astype(np.uint8)

        # Save the preprocessed image
        cv2.imwrite(output_path, output_image)
    except Exception as e:
        print(f"Error processing image {input_path}: {e}")

# Iterate through the input dataset
for folder_name in os.listdir(input_dir):
    folder_path = os.path.join(input_dir, folder_name)

    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Create the corresponding output folder
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        # Process each image in the folder
        for image_name in tqdm(os.listdir(folder_path), desc=f"Processing {folder_name}"):
            input_image_path = os.path.join(folder_path, image_name)
            output_image_path = os.path.join(output_folder_path, image_name)

            # Preprocess and save the image
            preprocess_and_save(input_image_path, output_image_path)

print("Preprocessing complete. Preprocessed images are saved in the 'preprocessed' folder.")


Processing bacterial_leaf_streak: 100%|██████████| 380/380 [00:02<00:00, 180.61it/s]

Preprocessing complete. Preprocessed images are saved in the 'preprocessed' folder.


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the saved model
model_path = "paddy_disease_model.h5"
model = load_model(model_path)
print(f"Model loaded from {model_path}")

# Class names (replace with the actual class names from training if needed)
class_names = [
    "bacterial_leaf_blight",
    "bacterial_leaf_streak",
    "bacterial_panicle_blight",
    "blast",
    "brown_spot",
    "dead_heart",
    "downy_mildew",
    "hispa",
    "normal",
    "tungro"
] # Replace with your actual class names

# Function to preprocess and predict for a single image
def predict_image(image_path, model, class_names, input_shape=(128, 128)):
    """
    Predicts the class of a single image using the loaded model.

    Args:
    - image_path (str): Path to the image file to predict.
    - model: Loaded Keras model.
    - class_names (list): List of class names corresponding to the model's outputs.
    - input_shape (tuple): Input shape for the model.

    Returns:
    - str: Predicted class name.
    """
    try:
        # Preprocess the image
        img = load_img(image_path, target_size=input_shape)
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        prediction = model.predict(img_array, verbose=0)
        predicted_class = class_names[np.argmax(prediction)]
        return predicted_class
    except Exception as e:
        return f"Error processing the image: {e}"

# Continuously take input images and predict
while True:
    # Input the path to the image
    image_path = input("Enter the path of the image to predict (or type 'exit' to quit): ").strip()
    if image_path.lower() == 'exit':
        print("Exiting prediction loop.")
        break

    # Check if the path exists
    if not os.path.exists(image_path):
        print("The provided image path does not exist. Please try again.")
        continue

    # Predict the class
    predicted_label = predict_image(image_path, model, class_names, input_shape=(128, 128))
    print(f"The model predicts the image is: {predicted_label}")


Model loaded from paddy_disease_model.h5
Enter the path of the image to predict (or type 'exit' to quit): \content\PADDY IMAGES\brown_spot\100035.jpg
The provided image path does not exist. Please try again.
Enter the path of the image to predict (or type 'exit' to quit): /content/PADDY IMAGES/brown_spot/100035.jpg
The model predicts the image is: brown_spot
Enter the path of the image to predict (or type 'exit' to quit): /content/PADDY IMAGES/tungro/100910.jpg
The model predicts the image is: blast
Enter the path of the image to predict (or type 'exit' to quit): /content/PADDY IMAGES/brown_spot/100905.jpg
The provided image path does not exist. Please try again.
Enter the path of the image to predict (or type 'exit' to quit): /content/PADDY IMAGES/tungro/100905.jpg
The model predicts the image is: brown_spot
Enter the path of the image to predict (or type 'exit' to quit): /content/PADDY IMAGES/tungro/100886.jpg
The model predicts the image is: tungro
Enter the path of the image to pre

KeyboardInterrupt: Interrupted by user

In [24]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Load the saved model
model_path = "/content/paddy_disease_model_100.h5"  # Adjust path to your model file in Colab
model = load_model(model_path)
print(f"Model loaded from {model_path}")

# Class names (replace with your actual class names)
class_names = [
    "bacterial_leaf_blight",
    "bacterial_leaf_streak",
    "bacterial_panicle_blight",
    "blast",
    "brown_spot",
    "dead_heart",
    "downy_mildew",
    "hispa",
    "normal",
    "tungro"
]

# Input directory containing subfolders for each class
input_dir = "/content/PADDY IMAGES"  # Adjust path to your "PADDY IMAGES" directory

# Function to preprocess and predict for a single image
def predict_image(image_path, model, class_names, input_shape=(128, 128)):
    try:
        # Preprocess the image
        img = load_img(image_path, target_size=input_shape)
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        prediction = model.predict(img_array, verbose=0)
        predicted_class = class_names[np.argmax(prediction)]
        return predicted_class
    except Exception as e:
        return f"Error: {e}"

# Collect all subdirectories (classes)
folders = [f for f in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, f))]

# Initialize counters
total_images = 0
correct_predictions = 0

# Process each folder
for folder in folders:
    folder_path = os.path.join(input_dir, folder)
    image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]

    # Randomly select 20 images or the maximum available if less than 20
    selected_images = random.sample(image_files, min(len(image_files), 20))

    for image_path in selected_images:
        # Predict the disease
        predicted_label = predict_image(image_path, model, class_names)

        # Compare with the folder name (actual disease)
        is_correct = predicted_label == folder
        correct_predictions += is_correct
        total_images += 1

        # Print details for each prediction
        print(f"Actual: {folder}, Predicted: {predicted_label}, Correct: {'Yes' if is_correct else 'No'}")

# Calculate accuracy
accuracy = (correct_predictions / total_images) * 100 if total_images > 0 else 0
print("\nSummary:")
print(f"Total Images: {total_images}")
print(f"Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy:.2f}%")


Model loaded from /content/paddy_disease_model_100.h5
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: blast, Correct: No
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: tungro, Correct: No
Actual: hispa, Predicted: dead_heart, Correct: No
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: hispa, Predicted: hispa, Correct: Yes
Actual: bacterial_leaf_